In [1]:
!pip install langchain-openai
!pip install langchain
!pip install python-dotenv
!pip install langchain_community
!pip install langchain_core
!pip install langchain-huggingface
!pip install streamlit
!pip install openai
!pip install langchain-groq
!pip install pypdf
!pip install PyPDF2
!pip install faiss-cpu
!pip install chromadb==0.6.1
!pip install bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4

## Imports

In [2]:
#from streamlit import streamlit as st
#from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader, PyPDFLoader, RecursiveUrlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from bs4 import BeautifulSoup
import os

# Kaggle specific imports 

from kaggle_secrets import UserSecretsClient
_cached_documents = None

In [3]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

# pull keys from kaggle secrets
os.environ['GROQ_API_KEY'] = user_secrets.get_secret('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = user_secrets.get_secret('LANGCHAIN_API_KEY')
#os.environ['HF_TOKEN'] = user_secrets.get_secret('HF_TOKEN')

In [4]:
# Initializing the model and embedding
llm = ChatGroq(model='Qwen-Qwq-32b')
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

2025-05-31 03:46:16.652224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748663176.832632      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748663176.886801      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Document Loaders Function Creation

In [5]:
# Load text documents

def load_documents(file_path):
    global _cached_documents 

    if _cached_documents is not None: 
        print('Using cached documents.')
        return _cached_documents

    documents=[] 
    for path in file_path: 
        if not os.path.exists(path):
            st.error(f"file not found: {path}")
            continue

        #load the document using text loader
        loader = TextLoader(path)
        documents.extend(loader.load())
    return documents

In [6]:
def load_pdf_documents(file_path):
    global _cached_documents 

    if _cached_documents is not None: 
        print('Using cached documents.')
        return _cached_documents

    documents=[] 
    for path in file_path: 
        if not os.path.exists(path):
            st.error(f"file not found: {path}")
            continue

        #load the document using PyPDFLoader
        loader = PyPDFLoader(path)
        documents.extend(loader.load())
    return documents

In [7]:
def load_web_documents(urls):
    global _cached_documents 

    if _cached_documents is not None: 
        print('Using cached documents.')
        return _cached_documents
        
    documents = []
    for url in urls:
        try:
            # Pages to scrape at 2 levels deep. 
            if "docs/integrations/" in url:
                loader = RecursiveUrlLoader(url, max_depth=2)
            # Pages to scrape at 1 level deep.
            elif "docs/how-to" in url or "docs/tutorials/" in url:
                loader = RecursiveUrlLoader(url, max_depth=1)
            # Pages to scrape at the direct link only
            else:
                loader = RecursiveUrlLoader(url, max_depth=0)
            documents.extend(loader.load())
        
        except Exception as  e:
            st.error(f"Error loading URL: {url}: {e}")
            
    return documents

## Functions to Split Documents and Create Vector Store

In [8]:
def split_documents(documents):
    global _cached_documents 

    if _cached_documents is not None: 
        print('Using cached documents.')
        return _cached_documents
        
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    return splits

In [9]:
def create_vector_store(splits):
    global _cached_documents 

    if _cached_documents is not None: 
        print('Using cached documents.')
        return _cached_documents

    vector_store = Chroma.from_documents(splits, embeddings, persist_directory='db')
    vector_store.persist()
    return vector_store

## List of Files / URLs to Load

In [10]:
txt_documents_to_load = [] # this is the text document to load, the r is used to indicate that the string is a raw string, so that the backslashes are not treated as escape characters.
pdf_documents_to_load = [] # list of pdf documents to load

recurive_base_urls_to_load = [
    "https://python.langchain.com/docs/how_to/document_loader_web/",
    "https://python.langchain.com/docs/integrations/document_loaders/",
    "https://python.langchain.com/docs/integrations/vectorstores/",
    "https://python.langchain.com/docs/integrations/retrievers/",
    "https://python.langchain.com/docs/integrations/text_embedding/",
    "https://python.langchain.com/docs/tutorials/chatbot/",
    "https://python.langchain.com/docs/tutorials/rag/",
    "https://python.langchain.com/docs/tutorials/qa_chat_history/"
]

direct_pages_to_load = [
    "https://python.langchain.com/docs/integrations/vectorstores/chroma",
    "https://python.langchain.com/docs/integrations/retrievers/contextual_compression",
]

In [11]:
#### concatenate all into a single list
web_urls_to_load = [] 
web_urls_to_load.extend(recurive_base_urls_to_load)
web_urls_to_load.extend(direct_pages_to_load)

In [12]:
kaggle_data_path = "/kaggle/input/llm-and-prompting-chatbot-data-for-data-scientists"

# Cheking the directory
if not os.path.exists(kaggle_data_path):
    print(f"Warning: Data directory not found at {kaggle_data_path}. Please ensure your dataset is correctly added.")

else:
    for file in os.listdir(kaggle_data_path):
        full_file_path = os.path.join(kaggle_data_path, file)
        if file.endswith(".pdf"):
            pdf_documents_to_load.append(full_file_path)
        elif file.endswith(".txt"): # Use 'elif' to avoid unnecessary checks
            txt_documents_to_load.append(full_file_path)

print(f"Found {len(pdf_documents_to_load)} PDF documents.")
print(f"Found {len(txt_documents_to_load)} TXT documents.")

Found 3 PDF documents.
Found 1 TXT documents.


In [13]:
# Load all form files into a single batch

all_documents = [] 
# all_documents.extend(load_documents(txt_documents_to_load)) # the file is currently empty, it will generate an error. 
all_documents.extend(load_pdf_documents(pdf_documents_to_load))
all_documents.extend(load_web_documents(web_urls_to_load))

print(f"Total Count: {len(all_documents)} documents.")

Total Count: 1049 documents.


## Split / Vectorize / Retrieve the Documents

In [14]:
splits = split_documents(all_documents)

In [15]:
vector_store = create_vector_store(splits)

/tmp/ipykernel_20/2754935198.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [16]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3}) # k is the number of documents to retrieve

## Creating a Chain with Retriever and Memory

In [17]:
def create_chain():
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

    #custom prompt template
    custom_prompt=PromptTemplate(
        input_variables = ["chat_history", "question", "context"],
        template=(
            "You are a helpful Q&A asistant focusing on Data Science and Machine Learning. Provide concise accurate answers"
            "Always provide only the direct answer Never include reasoning steps or any internal tags like <think>."
            "remove all internal tags prior to returning the answer."
            "If the answer is not in the context, say 'I don't know'.\n\n"
            "use the following context to answer the question:\n\n {context}\n\n" \
            "Question: {question}\n\nAnswer:"
        )
    )

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff", # "stuff" chain type is used for simple retrieval.
        retriever=retriever, 
        memory=memory, # this is used to store the chat history
        return_source_documents=True, # this returns the source documents for the answer. False means it will not return the source document for the answer. instead it will return the answer only.
        combine_docs_chain_kwargs = {"prompt": custom_prompt},  # this is used to set the prompt for the chain
    )
    return chain
    